In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.functions import col, expr, lit, min, max, mean

In [2]:
spark = SparkSession.builder\
    .config("spark.jars", "duckdb.jar") \
    .getOrCreate()

24/04/25 09:04:28 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [5]:
districtes = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", '''
        SELECT DISTINCT b.District 
        FROM rent_price b
    ''') \
    .load()

districtes = [i[0] for i in districtes.collect()]

In [6]:
districtes

['Eixample',
 'Sarria-Sant_Gervasi',
 'Sant_Marti',
 'Ciutat_Vella',
 'Les_Corts',
 'Sant_Andreu',
 'Gracia',
 'Horta-Guinardo',
 'Nou_Barris',
 'Sants-Montjuic']

In [27]:
for districte in districtes:
    DF = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:duckdb:trusted_zone/freshdata_trusted.db") \
    .option("driver", "org.duckdb.DuckDBDriver") \
    .option("query", f'''
        SELECT AVG(b.Price) as Price
        FROM rent_price b
        WHERE b.District = '{districte}'
        GROUP BY b.Trimester, b.Year
        ORDER BY b.Year, b.Trimester
    ''') \
    .load()
    print('fet', districte)
    DF.show()

fet Eixample
+------------------+
|             Price|
+------------------+
| 385.7950000000001|
|385.78833333333336|
| 395.2566666666667|
| 398.5233333333333|
|          401.5625|
| 412.3791666666666|
| 428.6666666666667|
| 427.2774999999999|
|441.01416666666665|
|444.86999999999995|
| 470.6258333333333|
|474.50416666666666|
| 477.8341666666667|
|494.38000000000005|
|505.99166666666656|
| 503.2416666666666|
|498.48166666666674|
|511.79333333333335|
| 526.2083333333333|
| 539.9799999999999|
+------------------+
only showing top 20 rows

fet Sarria-Sant_Gervasi
+------------------+
|             Price|
+------------------+
|491.32500000000005|
| 513.8249999999999|
|           527.945|
| 578.7441666666666|
|           538.785|
| 523.3516666666667|
| 582.8333333333334|
| 547.7858333333332|
| 560.3941666666667|
| 595.6183333333335|
| 618.6225000000001|
| 626.6475000000002|
| 625.5291666666666|
|          641.1375|
| 687.2275000000001|
| 640.8249999999999|
|           612.645|
| 611.6041666

In [ ]:
# Dividir datos en entrenamiento y prueba
train_size = int(len(data) * 0.9)  # Usaremos el 70% de los datos como entrenamiento
train, test = data[:train_size], data[train_size:]

# Configurar y ajustar el modelo ARIMA automáticamente
model = auto_arima(train, start_p=0, start_q=0, max_p=3, max_q=3, seasonal=False, trace=True)
model.fit(train)

# Hacer predicciones
predictions = model.predict(n_periods=len(test))

# Calcular el error
error = np.sqrt(mean_squared_error(test, predictions))

print(f'Predicción del siguiente valor: {model.predict(n_periods=1)[0]:.2f}')
print(f'Error RMSE en conjunto de prueba: {error:.2f}')

# Visualizar resultados
plt.figure(figsize=(10, 6))
plt.plot(data, label='Serie Original')
plt.plot(np.arange(train_size, len(data)), predictions, label='Predicciones', linestyle='--')
plt.title(f'AutoARIMA Prediction (RMSE={error:.2f})')
plt.xlabel('Índice de Tiempo')
plt.ylabel('Valor')
plt.legend()
plt.show()
